# Falabella

In [1]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
df_ml.dropna(subset=["tweet_text"], inplace=True)
df_ml = df_ml[df_ml["sentiment"] != "others"]
df_ml = df_ml.sample(n=100)
# df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
# df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")

In [2]:
def df_to_list(df):
    texts = df["tweet_text"]
    data = []
    for text in texts:
        data.append(text)
    return data

In [3]:
import numpy as np
import torch
from transformers import BertModel, BertTokenizer
tokenizer_español = BertTokenizer.from_pretrained("../pytorch/", do_lower_case=False)
model = BertModel.from_pretrained("../pytorch")

Some weights of the model checkpoint at ../pytorch were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from src import Features
inputs = df_to_list(df_ml)
inputs = Features.bert_encoder(inputs, tokenizer_español)
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [5]:
max_poll = Features.max_polling(last_hidden_states)
mean_poll = Features.mean_polling(last_hidden_states)

In [6]:
data_ml = {
        "max_polling_std": Features.std_normalization(max_poll),
        "max_polling_z_score": Features.z_score_normalization(max_poll),
        "max_polling_max": Features.min_max_normalization(max_poll),
        "mean_polling_std": Features.std_normalization(mean_poll),
        "mean_polling_z_score": Features.z_score_normalization(mean_poll),
        "mean_polling_max": Features.min_max_normalization(mean_poll),
    }

In [7]:
def find_indexs(list, value):
    list_aux = list.copy()
    output = []
    while True:
        try:
            index = list_aux.index(value)
            output.append(index)
            list_aux[index] = None
        except ValueError:
            break
    return output

# agrupamiento

# k-means

In [8]:
# k-means
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
agrupamiento = {}
metric = []
range_n_clusters = range(2, 8)
VP_by_k_centroids = {}
silhouettes_by_k_centroids = {}
for n_clusters in range_n_clusters:
    metrica = []
    for key, value in data_ml.items():
        kmeans = KMeans(
            n_clusters=n_clusters,
            init="random",
            random_state=1,
            n_init="auto",
        )
        cluster_labels = kmeans.fit_predict(value)
        # Labeling the clusters
        centers = kmeans.cluster_centers_

        silhouette_avg = silhouette_score(value, cluster_labels)
        silhouettes_by_k_centroids[key] = silhouette_avg

        variances = []
        for k in range(n_clusters):
            variance = Features.cluster_variance(value, cluster_labels, centers, k)
            variances.append(variance)
        avg_variance = sum(variances)/len(variances)
        VP_by_k_centroids[key] = avg_variance

/home/unknownsystem/Public/tt-pregrado/env_tt/lib/python3.8/site-packages/Tools-0.0.0-py3.8.egg/src/features/build_features.py:181: RuntimeWarning: invalid value encountered in double_scalars
/home/unknownsystem/Public/tt-pregrado/env_tt/lib/python3.8/site-packages/Tools-0.0.0-py3.8.egg/src/features/build_features.py:181: RuntimeWarning: invalid value encountered in double_scalars
/home/unknownsystem/Public/tt-pregrado/env_tt/lib/python3.8/site-packages/Tools-0.0.0-py3.8.egg/src/features/build_features.py:181: RuntimeWarning: invalid value encountered in double_scalars
/home/unknownsystem/Public/tt-pregrado/env_tt/lib/python3.8/site-packages/Tools-0.0.0-py3.8.egg/src/features/build_features.py:181: RuntimeWarning: invalid value encountered in double_scalars
/home/unknownsystem/Public/tt-pregrado/env_tt/lib/python3.8/site-packages/Tools-0.0.0-py3.8.egg/src/features/build_features.py:181: RuntimeWarning: invalid value encountered in double_scalars
/home/unknownsystem/Public/tt-pregrado/e

In [9]:
i=0
for key, value in data_ml.items():
    kmeans = KMeans(
        n_clusters=4,
        init="random",
        max_iter=100,
        random_state=rs[i][0],
        n_init=1,
    ).fit(value)
    agrupamiento[key] = list(kmeans.labels_)
    i += 1
clusters = {}
for key, value in agrupamiento.items():
    clusters[key] = {
    "cluster0": df_ml.iloc[find_indexs(agrupamiento[key], 0)],
    "cluster1": df_ml.iloc[find_indexs(agrupamiento[key], 1)],
    "cluster2": df_ml.iloc[find_indexs(agrupamiento[key], 2)],
    "cluster3": df_ml.iloc[find_indexs(agrupamiento[key], 3)],
}
# preprocesado vs sentimiento.
print("rs rsScore cluster 1 cluster 2 cluster 3 cluster 4")
i=0
for key0, value0 in clusters.items():
    data = []
    for key1, value1 in value0.items():
        sentiment = dict(value1["sentiment"].value_counts())
        sentiment_values = list(sentiment.values())
        sentiment_labels = list(sentiment.keys())
        sentiment = [f"{label} ({value})" for label, value in zip(sentiment_labels, sentiment_values)]
        data.append(sentiment)
    string = f"{rs[i][0]} & {rs[i][1]} & {data[0]} & {data[1]} & {data[2]} & {data[3]}"
    string = string.replace('[', '')
    string = string.replace(']', '')
    string = string.replace("'", "")
    print(string)
    i += 1

NameError: name 'rs' is not defined

# Extracción de información

## Modelo TCRE 

In [ ]:
# Example usage
corpus = []
key_list = list(clusters.keys())
for _, value0 in clusters[key_list[0]].items():
    corpus.append(value0['tweet_text'].tolist())
corpus[0]

['compre una silla p gamer para regalo de navidad y cometi el error en confiar que el producto venia en buenas condiciones y lamentablemente venia hasta con los pernos oxidados sucia etc.fue francamente decepcionante para mi hija. pesimo',
 'por favor, necesito ayuda, esperonun regalo de navidad que debio llegar el 17/12 y aun no llega, al preguntar, nadie sabe dar una respuesta, es aungustiante',
 'ojala que todo el retail manana donara juguetes, seria un gesto de humanidad para los ninos de la tragedia',
 ', cuantos meses mas debo esperar por mi polera? tres meses es mas que suficiente!!!',
 'la ayuda q me brindaron fue una escueta respuesta y cerraron la comunicacion sin poder responder y ahora hacer todo otra vez x vez numero 10. nefasto el postventa. vayanse a la falalalala lala lala los denunciare',
 'por que sus vendedores y jefes de los vendedores piensan que son duenos de la sucursal? y ellos de forma unilateral escogen los medios de pago. hoy fui a falabella de vina el cual t

In [ ]:
grupo = 0
corpus0 = []
cluster_result = []
for text_list in corpus:
    for text in text_list:
        corpus0.append(text)
        cluster_result.append(grupo)
    grupo += 1

In [ ]:
from src import Summarize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer = CountVectorizer()
tcre_results = Summarize.TCRE(corpus0, cluster_result, vectorizer)
print(tcre_results)

[['si', 'mas', 'ustedes', 'atencion', 'hagan', 'mensaje', 'plazo', 'buenas', 'regalo', 'compras', 'mal', 'venia', 'falabella', 'queremos', 'necesito', 'laburen', 'sistema', 'devolucion', 'cupo', 'vendedores', 'lala', 'vez', 'despacho', 'dinero', 'ir', 'meses', 'dias', 'puedo', 'online', 'ofrecen', 'pasean', 'pasen', 'pedi', 'pedido', 'perdere', 'perdi', 'pega', 'peor', 'paseado', 'perdieron', 'pequena', 'perder', 'ocultare', 'peores', 'pasa', 'pasaron', 'otro', 'oprimo', 'oxidados', 'pagina', 'pago', 'papa', 'op', 'papel', 'paquete', 'para', 'parece', 'pernos', 'parecia', 'pareja', 'ollas', 'ojala', 'otra', 'pasan', 'pasar', 'pascueros', 'pero', '10', 'pesar', 'preguntarle', 'preguntas', 'premium', 'prenda', 'preparaba', 'preparando', 'presencial', 'prestan', 'primera', 'privado', 'problema', 'problemas', 'procedimientos', 'procesos', 'producto', 'productos', 'progre', 'progrerio', 'promocion', 'preguntar', 'pregunta', 'precio', 'postventa', 'pesima', 'pesimo', 'pesos', 'piden', 'piens

In [ ]:
tweets = [s.replace(',', ' ').replace('.', ' ') for s in corpus[0]]
tweets = [s.rstrip('.').rstrip(',') + '.' for s in tweets]
tweets = [s.rstrip().rstrip('.') + '.' if s.endswith(' ') else s + '.' for s in tweets]


In [ ]:
from nltk.tokenize import sent_tokenize

#sentences = sent_tokenize(cluster_concatenated)
A = vectorizer.fit_transform(tweets).toarray()
A = A.T
tweets

['compre una silla p gamer para regalo de navidad y cometi el error en confiar que el producto venia en buenas condiciones y lamentablemente venia hasta con los pernos oxidados sucia etc fue francamente decepcionante para mi hija  pesimo..',
 'por favor  necesito ayuda  esperonun regalo de navidad que debio llegar el 17/12 y aun no llega  al preguntar  nadie sabe dar una respuesta  es aungustiante..',
 'ojala que todo el retail manana donara juguetes  seria un gesto de humanidad para los ninos de la tragedia..',
 '  cuantos meses mas debo esperar por mi polera? tres meses es mas que suficiente!!!..',
 'la ayuda q me brindaron fue una escueta respuesta y cerraron la comunicacion sin poder responder y ahora hacer todo otra vez x vez numero 10  nefasto el postventa  vayanse a la falalalala lala lala los denunciare..',
 'por que sus vendedores y jefes de los vendedores piensan que son duenos de la sucursal? y ellos de forma unilateral escogen los medios de pago  hoy fui a falabella de vina

In [ ]:
lsa_score_indices = Summarize.LSA_summary(A, 10)
lsa_results = [tweets[idx] for idx in lsa_score_indices]
for sentence in lsa_results:
    print(sentence)

por que sus vendedores y jefes de los vendedores piensan que son duenos de la sucursal? y ellos de forma unilateral escogen los medios de pago  hoy fui a falabella de vina el cual tengo un gran cupo con cmr y cupo con metodo de pago cheque y    ..
me cagaron la navidad y la sorpresa para mi hijo    me mandan un paquete con una bici que compre y resulta que vienen con los datos correctos y cuando la preparaba para dejarla en el arbol    me doy cuenta que no es la que pedi..
hay algo peor que el servicio a cliente de falabella?  nunca saben nada y atienden horrible  llevo mas de una semana tratando de saber cuando me llegara lo comprado  y esta gente de falabella no tienen respuesta  asi que no me queda otra que poner demanda a sernac..
pueden dejar de acosarme con sus llamadas  ya me tiene chata que con distintos numeros me siguan llamando a pesar de decir que no soy la persona que buscan  mas encima lo hacen con una ia que con suerte escucha ..
mi pareja y yo queremos el regalo de nues